# Machine Learning Nanodegree

## Capstone Project: Instacart Market Basket Analysis
### Which products will an Instacart consumer purchase again?

The dataset for this challenge is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order. The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order. We also provide the week and hour of day the order was placed, and a relative measure of time between orders. For more information, see the blog post accompanying its public release.

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Data Exploration
* [Step 2](#step2): Exploratory Visualizations
* [Step 3](#step3): Preprocessing 
* [Step 4](#step4): Benchmarks
* [Step 5](#step5): Algorithm and Techniques
* [Step 6](#step6): Refinements
* [Step 7](#step7): Algorithm Evaluation and Validation

---
<a id='step0'></a>
## Step 0: Import Datasets

In [ ]:
### Import libraries XXXXXXXXXXXXXXXXXXXX
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

%matplotlib inline

In [9]:
### Import Instacart Data
order_products_train_df = pd.read_csv("instacart_2017_05_01/order_products__train.csv")
order_products_prior_df = pd.read_csv("instacart_2017_05_01/order_products__prior.csv")
orders_df = pd.read_csv("instacart_2017_05_01/orders.csv")
products_df = pd.read_csv("instacart_2017_05_01/products.csv")
aisles_df = pd.read_csv("instacart_2017_05_01/aisles.csv")
departments_df = pd.read_csv("instacart_2017_05_01/departments.csv")

print('Total no. of orders: {}'.format(orders_df.shape[0]))
print('Total no. of products: {}'.format(products_df.shape[0]))
print('Total no. of aisles: {}'.format(aisles_df.shape[0]))
print('Total no. of departments: {}'.format(departments_df.shape[0]))

Total no. of orders: 3421083
Total no. of products: 49688
Total no. of aisles: 134
Total no. of departments: 21


---
<a id='step1'></a>
## Step 1: Data Exploration

orders_df tells to which set (prior, train, test) an order belongs. Will be predicting reordered items only for the 'test' set orders. 

In [4]:
orders_df.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [5]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [23]:
order_cnt = orders_df.groupby("eval_set").user_id.aggregate({'total_user':'nunique'}).reset_index()
order_cnt

,eval_set,total_user
0,prior,206209
1,test,75000
2,train,131209


These data frames (order_products_[prior/train]_df specify which products were purchased in each order. order_products_prior_df contains previous order contents for all customers. 

'reordered' indicates that the customer has a previous order that contains the product. (Some orders will have no reordered items). We may predict an explicit 'None' value for orders with no reordered items. 

In [15]:
order_products_prior_df.describe()

,order_id,product_id,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


In [18]:
order_products_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [16]:
order_products_train_df.describe()

,order_id,product_id,add_to_cart_order,reordered
count,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06
mean,1.706298e+06,2.555624e+04,8.758044e+00,5.985944e-01
std,9.897326e+05,1.412127e+04,7.423936e+00,4.901829e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.433700e+05,1.338000e+04,3.000000e+00,0.000000e+00
50%,1.701880e+06,2.529800e+04,7.000000e+00,1.000000e+00
75%,2.568023e+06,3.794000e+04,1.200000e+01,1.000000e+00
max,3.421070e+06,4.968800e+04,8.000000e+01,1.000000e+00


In [17]:
order_products_train_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [6]:
products_df.describe()

,product_id,aisle_id,department_id
count,49688.000000,49688.000000,49688.000000
mean,24844.500000,67.769582,11.728687
std,14343.834425,38.316162,5.850410
min,1.000000,1.000000,1.000000
25%,12422.750000,35.000000,7.000000
50%,24844.500000,69.000000,13.000000
75%,37266.250000,100.000000,17.000000
max,49688.000000,134.000000,21.000000


In [32]:
print(products_df.head())
products_df.tail()

   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13  


,product_id,product_name,aisle_id,department_id
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8
49687,49688,Fresh Foaming Cleanser,73,11


In [10]:
aisles_df.describe()

,aisle_id
count,134.000000
mean,67.500000
std,38.826537
min,1.000000
25%,34.250000
50%,67.500000
75%,100.750000
max,134.000000


In [33]:
print(aisles_df.head())
print(aisles_df.tail())

   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation
     aisle_id                       aisle
129       130    hot cereal pancake mixes
130       131                   dry pasta
131       132                      beauty
132       133  muscles joints pain relief
133       134  specialty wines champagnes


In [12]:
departments_df.describe()

,department_id
count,21.000000
mean,11.000000
std,6.204837
min,1.000000
25%,6.000000
50%,11.000000
75%,16.000000
max,21.000000


In [30]:
departments_df

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


Since products, aisles and departments data frames could be related amongst themselves using IDs as keys, we can merge them recursively and create a single data frame (pad_df for products, aisles and dept) for simplification as below:

In [22]:
pad_df = pd.merge(left=pd.merge(left=products_df, right=departments_df, how='left'), right=aisles_df, how='left')
pad_df.head()

,product_id,product_name,aisle_id,department_id,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,2,All-Seasons Salt,104,13,pantry,spices seasonings
2,3,Robust Golden Unsweetened Oolong Tea,94,7,beverages,tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen,frozen meals
4,5,Green Chile Anytime Sauce,5,13,pantry,marinades meat preparation


<a id='step2'></a>
## Step 2: Exploratory Vizualizations

In [25]:
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)#, color=color[2])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Maximum order number', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

NameError: name 'cnt_srs' is not defined

---
<a id='step3'></a>
## Step 3: Data Preprocessing


### Pre-process the Data

Starting with sorting the values based on user_id and order_number for the user.

In [27]:
orders_df.sort_values(by=['user_id', 'order_number'], inplace=True)
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [28]:
#Replace NaN with mean
orders_df.days_since_prior_order.fillna(orders_df.days_since_prior_order.mean(), inplace=True)
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,11.114836
1,2398795,1,prior,2,3,7,15.000000
2,473747,1,prior,3,3,12,21.000000
3,2254736,1,prior,4,4,7,29.000000
4,431534,1,prior,5,4,15,28.000000


---
<a id='step4'></a>
## Step 4: Benchmarks

---
<a id='step5'></a>
## Step 5: Algorithm and Techniques

PCA?

---
<a id='step6'></a>
## Step 6: Model Refinements


---
<a id='step7'></a>
## Step 7: Model Evaluation and Validation
